# Preamble

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
#from scipy.interpolate import interp1d
#from scipy.integrate import trapz
#from scipy.integrate import quad
#from scipy.optimize import fsolve

from utils.my_units import *

In [5]:
import matplotlib.pyplot as plt
import matplotlib as mpl
plt.rcdefaults()
from matplotlib import font_manager
from matplotlib import rcParams
from matplotlib import rc
from matplotlib import colors
rcParams['mathtext.rm'] = 'Times New Roman' 
rcParams['text.usetex'] = True
rcParams['font.family'] = 'times' #'sans-serif'
font_manager.findfont('serif', rebuild_if_missing=True)
fontsize = 14
rcParams.update({'font.size':fontsize})

In [4]:
from superrad import ultralight_boson as ub

# GW frequency

In [ ]:
bc = ub.UltralightBoson(spin=1, model="relativistic") 

In [18]:
int(len(df_duoblets)/2)

19

In [24]:
for i in range(int(len(df_duoblets))):
    if i%2==0:
        print(np.abs((df_duoblets.iloc[i]['F0'] - df_duoblets.iloc[i+1]['F0'])))

0.11240799999998785
0.0822430000000054
0.0
0.08451100000002043
0.0
0.14518400000000042
0.10005099999997924
0.14338300000002846
0.09299599999997099
0.0022840000000030614
0.042258000000003904
0.11934000000002243
0.057952000000000226
0.033742999999958556
0.01106300000000715
0.05068099999999731
0.12327700000000164
0.0
0.10132899999999267


### Frequency doubles

In [10]:
df_duoblets = pd.read_csv('data/pulsar_doublets.csv', index_col=0)

In [47]:
### Determine the range of alpha allowed for each pulsar
mbh_min, mbh_max = 3*MSolar, 100*MSolar
abh0 = 0.5
alpha_points_sol = 100
t_cutoff = 1e3*Year
alpha_points = 50

mu_list = 2*np.pi*df_duoblets['F0'].to_numpy()*Hz
alpha_list = np.nan+np.zeros((len(df_duoblets), alpha_points))

for i in tqdm(range(len(df_duoblets))):
    
    mu = mu_list[i]
    alpha_cutoff = 1.5*np.power(17*mu*0.1*t_cutoff, -1/10) ## take a factor of 1.5 greater than the estimated max alpha value
    alpha_min = GN*mu*mbh_min
    alpha_max = min(alpha_cutoff, GN*mu*mbh_max)
    
    if(alpha_max > alpha_min):
        
        alpha_list_temp = np.logspace(np.log10(alpha_min), np.log10(alpha_max), alpha_points_sol)
        gw_time_list_temp = np.nan+np.zeros(alpha_points_sol)
        
        for i_a, alpha in enumerate(alpha_list_temp):

            mbh = alpha/(mu*GN)/MSolar
            try:
                wf = bc.make_waveform(mbh, abh0, alpha, units="physical+alpha")
            except ValueError:
                break 

            if wf.azimuthal_num()==1:
                gw_time_list_temp[i_a] = wf.gw_time()*Second
               
        alpha_cutoff = 10**np.interp(0, np.flipud(np.log10(gw_time_list_temp/t_cutoff)), np.flipud(np.log10(alpha_list_temp)), left=np.nan, right=np.nan)
        alpha_list[i] = np.logspace(np.log10(alpha_min), np.log10(alpha_cutoff), alpha_points) 

100%|██████████| 38/38 [00:04<00:00,  8.58it/s]


In [48]:
np.min(alpha_list, axis=1), np.max(alpha_list, axis=1)

(array([0.01705474, 0.01706513, 0.03081223, 0.03081983, 0.01185086,
        0.01185086, 0.02287822, 0.02288603, 0.02081908, 0.02081908,
        0.01920352, 0.01921694, 0.01660172, 0.01661096, 0.02196837,
        0.02198163, 0.02471569, 0.02472429, 0.02029592, 0.02029613,
        0.00954534, 0.00954925, 0.01816045, 0.01817148, 0.0251416 ,
        0.02514696, 0.02589263, 0.02589575, 0.01805404, 0.01805506,
        0.03265846, 0.03266314, 0.0121308 , 0.0121422 , 0.03176519,
        0.03176519, 0.01303762, 0.01304698]),
 array([0.03658609, 0.03658371, 0.03435448, 0.03435358, 0.03803633,
        0.03803633, 0.03545903, 0.03545774, 0.0358168 , 0.0358168 ,
        0.03612628, 0.03612359, 0.03669127, 0.03668909, 0.03561249,
        0.03561021, 0.03516882, 0.03516751, 0.03591397, 0.03591393,
        0.03892743, 0.03892573, 0.03634191, 0.03633956, 0.03510498,
        0.03510419, 0.0349953 , 0.03499485, 0.03636464, 0.03636443,
        0.0341429 , 0.03414238, 0.03794153, 0.03793772, 0.03424355,
  

In [49]:
df_duoblets['min_alpha'] = np.min(alpha_list, axis=1)
df_duoblets['max_alpha'] = np.max(alpha_list, axis=1)